In [143]:
import pandas as pd

uid_check = ['아파트1-104-1006']
tss_transition = pd.DataFrame(columns=['K','tss','wss', 'ecv', 'cdpv', 'mdis' ,'mcdpv', 'og_length', 'new_length', 'sequence'])

In [146]:
import pandas as pd
from modules.UCKMeans import UCKMeans as UC
from modules.UCOSKMeans import UCOSKMeans as UCOS
from modules.SortMeans import SortMeans as SM

# Mongo 연결
from pymongo import MongoClient as mc
import random
import pprint as pp

#  print(uid_check)
# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
household_col = keti_pr_db.household_info

while True:
    total = household_col.find({"uid": { "$nin" : uid_check }}).count()
    db_datas = list(household_col.\
                    find({
                        "uid": { "$nin" : uid_check }
                        }).skip(random.randrange(0,total)).limit(1))
    if len(db_datas) == 0 :
        continue
    else:
        db_datas = db_datas[0]
        break;
        
# db_datas = household_col.\
#                 find_one({
#                     "uid": "아파트1-103-106"
#                 })

# require module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

sns.set(rc={'figure.figsize': (15.7, 13.27)})
plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import custom.supports as sp
import custom.mapgenerator as mg
import custom.dateutils as du

SEASONS = {
    "봄" : [3, 4, 5],
    "여름" : [6, 7, 8],
    "가을" : [9, 10, 11],
    "겨울" : [12, 1, 2]
}

SEASONSARR = ['봄','여름','가을','겨울']

'''
 0 : 봄
 1 : 여름
 2 : 가을
 3 : 겨울
'''

DAYARR = ["월", "화", "수", "목", "금", "토", "일"]

uid, timeslot = db_datas['uid'], db_datas['timeslot']
uid_check.append(uid)

print("{} pattern recognition".format(uid))


datelist = [
    dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
    for ts in timeslot
]
datelist = list(set(datelist))
datelist.sort()

ts_datas = {}
start_idx = 0
end_idx = 96
enl = 1

for date in datelist:
    ts_datas[date] = [ ts['power'] * enl for ts in timeslot[start_idx:end_idx]]
    start_idx = end_idx
    end_idx = end_idx + 96

ts_datas = pd.DataFrame(ts_datas).T
datas = ts_datas.reset_index().copy()

datas.rename(columns={"index": "date"}, inplace=True)
datas['date'] = pd.to_datetime(datas['date'])
datas['month'] = [dt.month for dt in datas['date']]
datas = [
    datas[(datas['month'].isin(SEASONS[season]))].copy()
    for season in SEASONSARR
]

bak_datas = datas.copy()

season_datas = {
    "봄": datas[0].copy(),
    "여름": datas[1].copy(),
    "가을": datas[2].copy(),
    "겨울": datas[3].copy()
}

# merge timeslot
season = "가을"

timeslot = 1
merge_size = timeslot * 4
merge_datas = season_datas[season].copy()
merge_datas = merge_datas[
    merge_datas.columns.difference(['month', 'date'])
]
merge_datas.index = season_datas[season]['date']

tmp = pd.DataFrame()
start_idx = 0
end_idx = 96
for date in merge_datas.index:
    in_arr = []
    og_arr = merge_datas.loc[date].values
    for merging in range(0, int(end_idx / merge_size)):
        in_arr.append(og_arr[
            (merging) * merge_size
            :
            (merging + 1) * merge_size
        ].sum())
#     logging
#     print("date end : {}, size: {}".format(date, len(in_arr)))

    tmp[date] = in_arr
merge_datas = tmp.copy()

merge_datas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
아파트2-3-303 pattern recognition


<ipython-input-146-c25bd3437e00>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


,2018-09-01,2018-09-02,2018-09-03,2018-09-04,2018-09-05,2018-09-06,2018-09-07,2018-09-08,2018-09-09,2018-09-10,...,2018-11-21,2018-11-22,2018-11-23,2018-11-24,2018-11-25,2018-11-26,2018-11-27,2018-11-28,2018-11-29,2018-11-30
0,0.156,0.203,0.143,0.105,0.103,0.205,0.196,0.117,0.129,0.127,...,0.188,0.267,0.166,0.203,0.159,0.202,0.207,0.224,0.180,0.165
1,0.132,0.232,0.113,0.114,0.118,0.157,0.136,0.120,0.115,0.131,...,0.176,0.215,0.259,0.173,0.236,0.202,0.173,0.179,0.249,0.255
2,0.121,0.173,0.106,0.103,0.135,0.110,0.146,0.110,0.167,0.116,...,0.176,0.175,0.243,0.154,0.162,0.169,0.170,0.167,0.158,0.180
3,0.116,0.103,0.099,0.113,0.116,0.111,0.123,0.122,0.173,0.133,...,0.216,0.167,0.169,0.157,0.164,0.189,0.222,0.206,0.173,0.172
4,0.110,0.106,0.115,0.104,0.112,0.106,0.120,0.126,0.116,0.130,...,0.159,0.163,0.164,0.158,0.171,0.258,0.162,0.191,0.161,0.159
5,0.116,0.108,0.109,0.108,0.105,0.115,0.127,0.115,0.128,0.201,...,0.165,0.165,0.184,0.156,0.216,0.173,0.166,0.168,0.159,0.156
6,0.125,0.106,0.105,0.111,0.113,0.112,0.159,0.128,0.100,0.233,...,0.156,0.151,0.164,0.163,0.099,0.162,0.177,0.169,0.159,0.172
7,0.133,0.151,0.146,0.130,0.111,0.126,0.178,0.164,0.118,0.165,...,0.154,0.144,0.163,0.203,0.171,0.162,0.204,0.179,0.361,0.236
8,0.107,0.137,0.204,0.148,0.164,0.212,0.187,0.178,0.103,0.226,...,0.364,0.330,0.322,0.211,0.247,0.310,0.287,0.253,0.198,0.273
9,0.084,0.094,0.090,0.191,0.289,0.089,0.221,0.099,0.155,0.229,...,0.238,0.064,0.198,0.312,0.261,0.183,0.186,0.101,0.080,0.144


In [147]:
from modules.THMeans import THMeans as tm
tm(merge_datas).run()

---Season Means---
---init TSS Check---
- calc TSS(Total Sum Of Squares) success!
---Dimension Reduction---
---Remove Outliers---
- dis_check: 0.6560654522381588, sim_check: 0.3854219781991462
- remove outlier success: 91 => 84
- calc TSS(Total Sum Of Squares) success!
---K Setting 6 ---
---6:Clustering Start---
---Now 1---
---First Cluster Group Init---
---Init Cluster---
---First K Is Mean Pattern---
---Rest K Select---
Test Start!
arr length: 84
Cluster Worst Pattern In
1: calc... [83, 0]
1: [83, 0, 42]
2: calc... [83, 42, 0]
2: calc... [83, 42, 0, 63]
2: [83, 42, 0, 63, 21]
4: calc... [83, 63, 42, 21, 0]
4: calc... [83, 63, 42, 21, 0, 73]
4: calc... [83, 63, 42, 21, 0, 73, 53]
4: calc... [83, 63, 42, 21, 0, 73, 53, 32]
4: [83, 63, 42, 21, 0, 73, 53, 32, 11]
-2: K Setting Okay
-3: K Setting Okay
-4: K Setting Okay
-5: K Setting Okay
-6: K Setting Okay
---Cluster Init Okay KMeans Start---
- calc WSS(Within cluster Sum Of Squares) success!
- calc ECV(Explained Cluster Value) success!


In [139]:
tm.divide_cluster_test()

Test Start!
arr length: 99
Cluster Worst Pattern In
1: calc... [98, 0]
1: [98, 0, 49]
2: calc... [98, 49, 0]
2: calc... [98, 49, 0, 74]
2: [98, 49, 0, 74, 25]
4: calc... [98, 74, 49, 25, 0]
4: calc... [98, 74, 49, 25, 0, 86]
4: calc... [98, 74, 49, 25, 0, 86, 62]
4: calc... [98, 74, 49, 25, 0, 86, 62, 37]
4: [98, 74, 49, 25, 0, 86, 62, 37, 13]
[98, 74, 49, 25, 0, 86, 62]
